In [42]:
import cPickle as pkl
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped
from cspython.analysis import roster_match
import cPickle as pkl
import sys
sys.setrecursionlimit(15000)
import pandas as pd
import itertools

In [2]:
with open('../cspython/BIG_2016-10-01_to_2017-10-13.pkl', 'rb') as f:
    d = pkl.load(f)
    
big_data = process_scrapped(d)
overview, big_data = big_data

In [3]:
cur_roster = ['gob b', 'LEGIJA', 'tabseN', 'nex', 'keev', 'kakafu'] 

In [19]:
m = roster_match(big_data, 'BIG', cur_roster)

In [26]:
total_matches = []
for idx in m.index:
    series = big_data[idx]
    if 'Dust2' not in series['map_pool']:
        total_matches.append(idx)

In [62]:
mp = d[0]['map_pool']

In [36]:
vm = create_veto_matrix(total_matches, big_data, mp, 'BIG')

In [37]:
vm.sum(1)

Cache          -31
Mirage          46
Inferno         32
Nuke           250
Train           13
Cobblestone     47
Overpass        12
dtype: int64

In [39]:
vm.to_excel('veto_comparison.xlsx')

In [55]:
new_vm = pd.DataFrame()
for row_map, col_map in itertools.product(vm.index, vm.index):
    score = vm.loc[col_map, row_map] - vm.loc[row_map, col_map]
    new_vm.loc[row_map, col_map] = score

In [100]:
new_vm.to_excel('map_preference.xlsx')

In [56]:
new_vm.sum(1)

Cache           96.0
Mirage           5.0
Inferno         26.0
Nuke          -228.0
Train           45.0
Cobblestone      5.0
Overpass        51.0
dtype: float64

In [57]:
new_vm.loc['sum',:] =  new_vm.sum()
new_vm.loc[:,'sum']=  new_vm.sum(1)

In [70]:
new_vm.loc[:, 'vetoed by BIG'] = (r/61).round(3)

In [99]:
new_vm.loc[:, 'picked by BIG'] = r_picked.round(3)

In [98]:
r_picked

Cache          0.300
Mirage         0.075
Inferno        0.150
Nuke           0.000
Train          0.150
Cobblestone    0.150
Overpass       0.175
dtype: float64

In [83]:
new_vm.to_excel('map_preference.xlsx')

In [27]:
def create_veto_matrix(matche_idxs, serieses, map_pool, team_name):
    veto_matrix = pd.DataFrame(index=map_pool, columns=map_pool).fillna(0)
    for idx in matche_idxs:
        vetos= serieses[idx]['vetos']
        remaining = list(map_pool)
        if vetos is None:
            continue
        for v in vetos:
            picked_map = False
            if 'left over' in v:
                continue
            if team_name not in v:
                remaining.remove(v.split(' ')[-1])
            else:
                if 'picked' in v:
                    picked_map = v.split('picked')[-1].strip()
                elif 'removed' in v:
                    removed_map = v.split('removed')[-1].strip()
                if picked_map:
                    remaining.remove(picked_map)
                    for m in remaining:
                        veto_matrix.loc[picked_map, m] -= 1
                else:
                    remaining.remove(removed_map)
                    for m in remaining:
                        veto_matrix.loc[removed_map, m] += 1

    return veto_matrix

In [64]:
team_name = 'BIG'
def num_times_vetoted(mp, d, team_name)
    r = pd.Series(index=mp).fillna(0)
    count = 0
    for s in d:
        if 'Dust2' in s['map_pool'] or s['vetos'] == []:
            continue
        maps_remaining = list(s['map_pool'])
        for v in s['vetos']:
            if 'left over' in v:
                continue
            if team_name not in v:
                maps_remaining.remove(v.split(' ')[len(v.split(' '))-1])
            else:
                if 'removed' in v:
                    map_vetoed = v.split(' ')[len(v.split(' '))-1]
                    r.loc[map_vetoed] += 1 
        count += 1
    return r/c

In [92]:

def num_times_selected(mp, d, team_name):
    r = pd.Series(index=mp).fillna(0)
    count = 0
    for s in d:
        if 'Dust2' in s['map_pool'] or s['vetos'] == []:
            continue
        maps_remaining = list(s['map_pool'])
        for v in s['vetos']:
            if 'left over' in v:
                continue
            if team_name not in v:
                maps_remaining.remove(v.split(' ')[len(v.split(' '))-1])
            else:
                if 'picked' in v:
                    map_picked = v.split(' ')[len(v.split(' '))-1]
                    r.loc[map_picked] += 1 
                    count += 1
    return r/count

In [93]:
r_picked = num_times_selected(mp, d, 'BIG')

In [96]:
r_picked

Cache          0.300
Mirage         0.075
Inferno        0.150
Nuke           0.000
Train          0.150
Cobblestone    0.150
Overpass       0.175
dtype: float64

In [91]:

team_name = 'BIG'
for s in d:

    if 'Dust2' in s['map_pool'] or s['vetos'] == []:
        continue
    print '##########################################'
    maps_remaining = list(s['map_pool'])
    for v in s['vetos']:
        print v
       
        if 'left over' in v:
            
            continue
        if team_name not in v:
            maps_remaining.remove(v.split(' ')[len(v.split(' '))-1])
        else:

            maps_remaining.remove(v.split(' ')[len(v.split(' '))-1])

##########################################
1. NiP removed Overpass
2. BIG removed Nuke
3. NiP picked Train
4. BIG picked Cache
5. NiP removed Cobblestone
6. BIG removed Inferno
7. Mirage was left over
##########################################
1. PRIDE removed Overpass
2. BIG removed Nuke
3. PRIDE picked Train
4. BIG picked Cache
5. PRIDE removed Inferno
6. BIG removed Mirage
7. Cobblestone was left over
##########################################
1. BIG removed Nuke
2. mousesports removed Overpass
3. BIG picked Cache
4. mousesports picked Cobblestone
5. BIG removed Inferno
6. mousesports removed Train
7. Mirage was left over
##########################################
1. Space Soldiers removed Inferno
2. BIG removed Nuke
3. Space Soldiers picked Cache
4. BIG picked Overpass
5. Space Soldiers removed Train
6. BIG removed Cobblestone
7. Mirage was left over
##########################################
1. BIG removed Nuke
2. EURONICS removed Mirage
3. BIG picked Cache
4. EURONICS picked Cobb